In [5]:
import folium
import requests
import json
from shapely.geometry import shape
import pandas as pd 

# Load the dataset
df_Heatmap = pd.read_csv("Cleaned_dataset.csv")

# Count the number of heart attacks for each state
heart_attack_counts = df_Heatmap[df_Heatmap['HadHeartAttack'] == 'Yes'].groupby('State').size().reset_index(name='count')

# Load GeoJSON file with US state boundaries
url = 'https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/geojson/us-states.json'
us_states_geo = requests.get(url).json()

# Manually add missing states with small polygons
additional_states = [
    {"type": "Feature", "properties": {"name": "District of Columbia"}, "geometry": {"type": "Polygon", "coordinates": [[[-77.042, 38.907], [-77.032, 38.907], [-77.032, 38.912], [-77.042, 38.912], [-77.042, 38.907]]]}},
    {"type": "Feature", "properties": {"name": "Guam"}, "geometry": {"type": "Polygon", "coordinates": [[[144.792, 13.442], [144.797, 13.442], [144.797, 13.447], [144.792, 13.447], [144.792, 13.442]]]}},
    {"type": "Feature", "properties": {"name": "Puerto Rico"}, "geometry": {"type": "Polygon", "coordinates": [[[-66.595, 18.219], [-66.585, 18.219], [-66.585, 18.224], [-66.595, 18.224], [-66.595, 18.219]]]}},
    {"type": "Feature", "properties": {"name": "Virgin Islands"}, "geometry": {"type": "Polygon", "coordinates": [[[-64.901, 18.334], [-64.891, 18.334], [-64.891, 18.339], [-64.901, 18.339], [-64.901, 18.334]]]}}
]

us_states_geo['features'].extend(additional_states)

# Create a dictionary to map state names to their initials
state_initials = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA', 'Colorado': 'CO',
    'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA',
    'Maine': 'ME', 'Maryland': 'MD', 'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH', 'New Jersey': 'NJ',
    'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH',
    'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA',
    'Washington': 'WA', 'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC',
    'Guam': 'GU', 'Puerto Rico': 'PR', 'Virgin Islands': 'VI'
}


# Save GeoJSON data to a file
with open('us_states_geo.json', 'w') as f:
    json.dump(us_states_geo, f)

# Save heart attack counts to a JSON file
heart_attack_counts.to_json('heart_attack_counts.json', orient='records')

# Function to calculate the centroid of polygons and multipolygons
def calculate_centroid(geometry):
    geom = shape(geometry)
    centroid = geom.centroid
    return centroid.y, centroid.x

# Function to create a choropleth map using Folium
def create_Heat_map(geo_data, data, state_initials):
    # Create a map centered on the USA
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)

    # Create choropleth map
    folium.Choropleth(
        geo_data=geo_data,
        name='choropleth',
        data=data,
        columns=['State', 'count'],
        key_on='feature.properties.name',
        fill_color='YlOrRd',
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name='Number of Heart Attacks',
        highlight=True
    ).add_to(m)
    
    # Add state borders and initials
    for feature in geo_data['features']:
        state_name = feature['properties']['name']
        if state_name in state_initials:
            initials = state_initials[state_name]
            geometry = feature['geometry']
            lat, lon = calculate_centroid(geometry)
            folium.Marker(
                location=[lat, lon],
                icon=folium.DivIcon(html=f'<div style="font-size: 10px; color: black;">{initials}</div>')
            ).add_to(m)
    
    # Add legend with initials and full names
    legend_html = '''
     <div style="position: fixed; 
     bottom: 200px; left: 10px; width: 150px; height: 250px; 
     background-color: white; border-radius: 5px; z-index:9999; font-size:10px;
     overflow-y: scroll; padding: 10px; border: 2px solid grey">
     <br>
     <h4 style="text-align:center; font-weight: bold; margin-top: 10px;">State Initials</h4>
     <div style="display: flex; flex-wrap: wrap;">
    '''
    for state, initials in state_initials.items():
        legend_html += f'<div style="margin: 5px;"><strong>{initials}</strong>: {state}</div>'
    legend_html += '</div></div>'
    
    m.get_root().html.add_child(folium.Element(legend_html))

    return m

# Convert state names in GeoJSON to match those in heart_attack_counts
for feature in us_states_geo['features']:
    feature['id'] = feature['properties']['name']

# Create the heatmap
heatmap = create_Heat_map(us_states_geo, heart_attack_counts, state_initials)

# Save to an HTML file
heatmap.save('us_heart_attack_heatmap.html')

# Display the map (if running in a Jupyter notebook)
heatmap
